In [ ]:
# default_exp models.pretrained.lstm

In [ ]:
# all_func


## fasta + BioPython

In [7]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

import pandas as pd

from peptide.basics import *
from peptide.preprocessing.data import (
    ProteinDataset,
    ACPDataset,
    AMPDataset,
    DNABindDataset,
)


In [2]:
record = SeqRecord(
    Seq("MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF"),
    id="YP_025292.1",
    name="HokC",
    description="toxic membrane protein, small",
)
print(record.format('fasta'))

>YP_025292.1 toxic membrane protein, small
MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF



In [5]:
record[:5]

SeqRecord(seq=Seq('MKQHK'), id='YP_025292.1', name='HokC', description='toxic membrane protein, small', dbxrefs=[])

In [8]:
acp_data = ACPDataset(DATA_STORE)
amp_data = AMPDataset(DATA_STORE)
dnabind_data = DNABindDataset(DATA_STORE)

In [10]:
def generate_fasta_files(ds: ProteinDataset, out_dir: str = None, use_seq_max_len: bool = False):
    """Generate a fasta files for the given protein dataset."""

    ds_name = type(ds).__name__
    location = out_dir if out_dir else ds.location

    for df, split in zip([ds.train, ds.test], ['train', 'test']):
        if use_seq_max_len:
            file_name = f"{location}/{ds_name}_{split}_seqlen_{ds.max_seq_len}.fasta"
            # create list of seq records, with truncated sequences
            seq_recs = [
                SeqRecord(
                    Seq("".join(df.loc[i, "sequence"])), id=str(i))
                    [:ds.max_seq_len] 
                    for i in range(len(df))
            ]
        else:
            file_name = f"{location}/{ds_name}_{split}.fasta"
            # create list of seq records, with full sequences
            seq_recs = [
                SeqRecord(Seq("".join(df.loc[i, "sequence"])), id=str(i))
                for i in range(len(df))
            ]

        # write fasta file
        with open(file_name, "w") as output_handle:
            SeqIO.write(seq_recs, output_handle, "fasta")

        print(
            f"Created {file_name} with {len(seq_recs)} sequence records"
        )


In [15]:
generate_fasta_files(acp_data)

Created /home/vinod/.peptide/datasets/ACPDataset_train.fasta with 1378 sequence records
Created /home/vinod/.peptide/datasets/ACPDataset_test.fasta with 344 sequence records


In [16]:
generate_fasta_files(amp_data)

Created /home/vinod/.peptide/datasets/AMPDataset_train.fasta with 3234 sequence records
Created /home/vinod/.peptide/datasets/AMPDataset_test.fasta with 808 sequence records


In [17]:
generate_fasta_files(amp_data, use_seq_max_len=True)

Created /home/vinod/.peptide/datasets/AMPDataset_train_seqlen_150.fasta with 3234 sequence records
Created /home/vinod/.peptide/datasets/AMPDataset_test_seqlen_150.fasta with 808 sequence records


In [18]:
generate_fasta_files(dnabind_data)

Created /home/vinod/.peptide/datasets/DNABindDataset_train.fasta with 14189 sequence records
Created /home/vinod/.peptide/datasets/DNABindDataset_test.fasta with 2272 sequence records


In [19]:
generate_fasta_files(dnabind_data, use_seq_max_len=True)

Created /home/vinod/.peptide/datasets/DNABindDataset_train_seqlen_300.fasta with 14189 sequence records
Created /home/vinod/.peptide/datasets/DNABindDataset_test_seqlen_300.fasta with 2272 sequence records


## Prose

Sample python commands to generate embeddings from pretrained model

**ACP**
```
python embed_sequences.py -o ~/.peptide/datasets/lstm/acp_nopool_train.h5 ~/.peptide/datasets/fasta/ACPDataset_train.fasta
# loading the pre-trained ProSE MT model
# writing: /home/vinod/.peptide/datasets/lstm/acp_nopool_train.h5
# embedding with pool=none

python embed_sequences.py --pool avg -o ~/.peptide/datasets/lstm/acp_avgpool_test.h5 ~/.peptide/datasets/fasta/ACPDataset_test.fasta
# loading the pre-trained ProSE MT model
# writing: /home/vinod/.peptide/datasets/lstm/acp_avgpool_test.h5
# embedding with pool=avg
```

**AMP**
- For AMP - some type of pooling needs to be done on the non-truncated sequences as `train` and `test` have different max seq lengths
```
python embed_sequences.py --pool avg -o ~/.peptide/datasets/lstm/amp_avgpool_train.h5 ~/.peptide/datasets/fasta/AMPDataset_train.fasta
# loading the pre-trained ProSE MT model
# writing: /home/vinod/.peptide/datasets/lstm/amp_avgpool_train.h5
# embedding with pool=avg
```
- And no pooling can only be done on truncated sequences since the lengths match
```
python embed_sequences.py -o ~/.peptide/datasets/lstm/amp_nopool_test_seqlen_150.h5 ~/.peptide/datasets/fasta/AMPDataset_test_seqlen_150.fasta
# loading the pre-trained ProSE MT model
# writing: /home/vinod/.peptide/datasets/lstm/amp_nopool_test_seqlen_150.h5
# embedding with pool=none
```

**DNA Binding**
- Same as AMP - some pooling needed for the full non-truncated sequences
```
python embed_sequences.py -o ~/.peptide/datasets/lstm/dnabind_nopool_train_seqlen_300.h5 ~/.peptide/datasets/fasta/DNABindDataset_train_seqlen_300.fasta
# loading the pre-trained ProSE MT model
# writing: /home/vinod/.peptide/datasets/lstm/dnabind_nopool_train_seqlen_300.h5
# embedding with pool=none
```
- And pooling can be done on the truncated sequences
```
python embed_sequences.py --pool avg -o ~/.peptide/datasets/lstm/dnabind_avgpool_test.h5 ~/.peptide/datasets/fasta/DNABindDataset_test.fasta
# loading the pre-trained ProSE MT model
# writing: /home/vinod/.peptide/datasets/lstm/dnabind_avgpool_test.h5
# embedding with pool=avg
```

## H5

In [ ]:
import h5py

In [ ]:
filename = "data/acp_test_avgpool.h5"

In [ ]:

with h5py.File(filename, "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    print("--")
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[1]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_arr = f[a_group_key][()]  # returns as a numpy array

Keys: <KeysViewHDF5 ['1', '2', '3', '4']>
--
<class 'h5py._hl.dataset.Dataset'>


In [ ]:
ds_arr.shape

(6165,)

In [ ]:
ds_arr

array([ 0.04545455,  0.04545455,  0.11363637, ...,  0.09409127,
       -0.05029093,  0.62466204], dtype=float32)

In [ ]:
f = h5py.File(filename, "r")

In [ ]:
for key in f.keys():
    print(f'indx: {key}') #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(f[key])) # get the object type: usually group or dataset
    data = f[key][()]
    print(data.shape)
f.close()


indx: 1
<class 'h5py._hl.dataset.Dataset'>
(6165,)
indx: 2
<class 'h5py._hl.dataset.Dataset'>
(6165,)
indx: 3
<class 'h5py._hl.dataset.Dataset'>
(6165,)
indx: 4
<class 'h5py._hl.dataset.Dataset'>
(6165,)


In [2]:
import torch

In [5]:
acp_train = torch.load('/home/vinod/.peptide/datasets/lstm/acp_nopool_train.h5')

UnpicklingError: invalid load key, 'H'.

y